In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Nov__3_21:07:56_CDT_2017
Cuda compilation tools, release 9.1, V9.1.85
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [1]:
cd mmrotate

/mnt/mars-beta/tochukwu/maestro/maestro_code/mmrotate


In [2]:
!pip3 install -U openmim

In [3]:
!mim install mmcv-full

/home/tochukwu/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html
Using cached `mminstall.txt` for mmdet==2.25.0: /home/tochukwu/.cache/mim/mminstall/mmdet==2.25.0.txt
Get 'mim' extra requirements from `mminstall.txt` for mmdet 2.25.0: ['mmcv-full>=1.3.17'].


In [4]:
!mim install mmdet

/home/tochukwu/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html
Using cached `mminstall.txt` for mmdet==2.25.0: /home/tochukwu/.cache/mim/mminstall/mmdet==2.25.0.txt
Get 'mim' extra requirements from `mminstall.txt` for mmdet 2.25.0: ['mmcv-full>=1.3.17'].


In [6]:
from mmcv import collect_env
collect_env()

# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

0.3.2
2.25.0
10.1
GCC 7.3


In [9]:
from mmrotate.datasets.builder import ROTATED_DATASETS
from mmrotate.datasets.dota import DOTADataset


@ROTATED_DATASETS.register_module()
class BodyCam(DOTADataset):
    """SAR ship dataset for detection."""
    CLASSES = ('Clipper', 'Crocodilegrasper', 'HookDiathermy', 'Marylandgrasper', 'Scissors', 'SurgicalInstrument', 'Trocar', 'Yohangrasper',)

In [5]:
cd mmrotate

/mnt/mars-beta/tochukwu/maestro/maestro_code/mmrotate


In [6]:
from mmcv import Config
cfg = Config.fromfile('./configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_dota_le90.py')

In [14]:
!wget https://download.openmmlab.com/mmrotate/v0.1.0/rotated_retinanet/rotated_retinanet_hbb_r50_fpn_1x_dota_oc/rotated_retinanet_hbb_r50_fpn_1x_dota_oc-e8a7c7df.pth

--2022-07-07 15:15:32--  https://download.openmmlab.com/mmrotate/v0.1.0/rotated_retinanet/rotated_retinanet_hbb_r50_fpn_1x_dota_oc/rotated_retinanet_hbb_r50_fpn_1x_dota_oc-e8a7c7df.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 8.208.41.29
Connecting to download.openmmlab.com (download.openmmlab.com)|8.208.41.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146852154 (140M) [application/octet-stream]
Saving to: ‘rotated_retinanet_hbb_r50_fpn_1x_dota_oc-e8a7c7df.pth’

rotated_retinanet_h 100%[===================>] 140.05M  11.8MB/s    in 12s     

2022-07-07 15:15:44 (11.7 MB/s) - ‘rotated_retinanet_hbb_r50_fpn_1x_dota_oc-e8a7c7df.pth’ saved [146852154/146852154]



In [7]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'BodyCam'
cfg.data_root = 'train/'

cfg.data.test.type = 'BodyCam'
cfg.data.test.data_root = 'train/'
cfg.data.test.ann_file = 'lableTxt'
cfg.data.test.img_prefix = 'Images'

cfg.data.train.type = 'BodyCam'
cfg.data.train.data_root = 'train/'
cfg.data.train.ann_file = 'lableTxt'
cfg.data.train.img_prefix = 'Images'

cfg.data.val.type = 'BodyCam'
cfg.data.val.data_root = 'train/'
cfg.data.val.ann_file = 'lableTxt'
cfg.data.val.img_prefix = 'Images'

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 8
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = '/mnt/mars-beta/tochukwu/maestro/maestro_code/mmrotate/rotated_retinanet_hbb_r50_fpn_1x_dota_oc-e8a7c7df.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

cfg.optimizer.lr = 0.001
cfg.lr_config.warmup = None
cfg.runner.max_epochs = 3
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 3
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'BodyCam'
data_root = 'train/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RResize', img_scale=(1024, 1024)),
    dict(
        type='RRandomFlip',
        flip_ratio=[0.25, 0.25, 0.25],
        direction=['horizontal', 'vertical', 'diagonal'],
        version='le90'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024),
        flip=False,
        transforms=[
            dict(type='RResize'),
            dict(
                type='Normaliz

In [16]:
pip install future tensorboard

  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached absl_py-1.1.0-py3-none-any.whl (123 kB)
  Using cached google_auth-2.9.0-py2.py3-none-any.whl (167 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached grpcio-1.47.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.5 MB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached Werkzeug-2.1.2-py3-none-any.whl (224 kB)
  Using cached protobuf-3.19.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl 

In [1]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot
from mmrotate.models import build_detector

In [10]:
import os.path as osp

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/home/tochukwu/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
2022-07-07 15:18:51,294 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-07-07 15:18:51,299 - mmdet - INFO - load checkpoint from local path: /mnt/mars-beta/tochukwu/maestro/maestro_code/mmrotate/rotated_retinanet_hbb_r50_fpn_1x_dota_oc-e8a7c7df.pth
2022-07-07 15:18:51,394 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for neck.lateral_convs.0.conv.weight: copying a param with shape torch.Size([256, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([256, 256, 1, 1]).
size mismatch for neck.lateral_convs.1.conv.weight: copying a param with shape torch.Size([256, 1024, 1, 1]) from checkpoint, the shape i

ValueError: need at least one array to concatenate